In [ ]:
import pandas as pd
import numpy as np
import os
import scipy.stats as stats
from utils import load_data, val_loglhood

# Gibbs Sampling

In [ ]:
def gibbs_sampling(iters, data_path, K, p, q, debug=False):
    """
    data_path: path where data is saved.
    K: number of plants (n_plants in load_data function).
    p: time interval to consider.
    q: sample distribution for parameters.
    """
    if debug:
        print('Loading data...')
    Y0, X = load_data(data_path, K, p, resample_rule='4H')
    if debug:
        print('Y0 shape: {}'.format(Y0.shape))
        print('X shape: {}'.format(X.shape))
    theta = init_parameters(K, p, q, Y0, X, debug)
    if debug:
        print('Parameters intialized!')
    samples = []
    for i in range(iters):
        if debug:
            print('Iteration {}'.format(i))
        for j in theta.shape[0]:
            while True:
                theta_j = q.rvs()
                lk = val_loglhood(theta, Y0, X, True)
                if lk != -np.inf:
                    break
        samples.append(theta)
    return np.array(samples)
        
    
def init_parameters(K, p, q, Y0, X, debug=False):
    if debug:
        print('Initializing parameters...')
    while True:
        theta = np.zeros(K ** 2 * (p + 1))
        for i in range(theta.shape[0]):
            theta[i] = q.rvs()

        # Force CovU to be positive semidefinite.
        covu = np.reshape(theta[-K**2:], (K, K)).T
        covu = np.dot(covu.T, covu)
        theta[-K**2:] = np.reshape(covu, K**2)
        
        lk = val_loglhood(theta, Y0, X, True)
        print('LK = {}'.format(lk))
        if lk != -np.inf:
            return theta
        

In [ ]:
a = np.arange(20)
print(a)
b = a[-3 ** 2:]
print(b)
c = np.reshape(b, (3, 3)).T
print(c)
print(np.dot(c.T, c))
d = np.reshape(b, 3 ** 2)
print(d)

In [ ]:
DATA_PATH = '/home/chrisams/Documents/data_TAIM/processed/'
q = stats.norm(loc=0, scale=1)
K = 2
p = 2
iters = 5
debug = True

In [ ]:
gibbs_sampling(iters, DATA_PATH, K, p, q, debug)